In [56]:
# подгружаем модуль re для работы с текстом
import re
import keras
from keras.preprocessing.text import Tokenizer
import numpy as np
from keras.layers import *
from keras.models import Sequential
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping

### Слова


In [43]:
with open('train_data.txt', 'r', encoding='utf-8') as f:
    texts = f.read()
    # убираем первый невидимый символ
    texts = texts.replace('\ufeff', '') 
    

In [44]:
# набор уникальных слов
maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, 
                      filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»',
                      lower=True,
                      split=' ',
                      char_level=False)
tokenizer.fit_on_texts([texts])

In [45]:
dist = list(tokenizer.word_counts.items())
print(dist[:10])

[('вы', 3), ('лучший', 1), ('ответ', 1), ('на', 1), ('проблемы', 1), ('которые', 1), ('возникли', 1), ('в', 3), ('понедельник', 2), ('думайте', 1)]


In [46]:
# преобразуем текст в последовательность чисел в соответсвии со словарем
data = tokenizer.texts_to_sequences([texts])

In [47]:
data
# в текст вместо каждого слова поставили цифру из токенайзера

[[1,
  4,
  5,
  6,
  7,
  8,
  9,
  2,
  3,
  10,
  11,
  12,
  13,
  2,
  14,
  15,
  16,
  17,
  18,
  19,
  1,
  20,
  2,
  3,
  21,
  22,
  23,
  24,
  1,
  25,
  26]]

In [48]:
len(data[0])

31

In [49]:
# категоризируем слова в вектор
res = keras.utils.to_categorical(data[0], num_classes=maxWordsCount)
print( res.shape )

(31, 1000)


(31, 1000): 31 - кол-во слов в тексте, 1000 - длина вектора на каждое слово (можно было сделать меньше, напр, 50 или 100)

In [50]:
# число слов, на которых строится прогноз
inp_words = 3
n = res.shape[0]-inp_words

In [51]:
X = np.array([res[i:i+inp_words, :] for i in range(n)])
Y = res[inp_words:] 

In [52]:
model = Sequential([
    Input((inp_words, maxWordsCount)),
    SimpleRNN(128, activation='tanh'),
    Dense(maxWordsCount, activation='softmax')
])

model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_2 (SimpleRNN)    (None, 128)               144512    
                                                                 
 dense_2 (Dense)             (None, 1000)              129000    
                                                                 
Total params: 273,512
Trainable params: 273,512
Non-trainable params: 0
_________________________________________________________________


In [53]:
model.compile(loss='categorical_crossentropy',
              metrics=['accuracy'],
              optimizer='adam')

In [59]:
# Настраиваем защиту от переобучения
estop = EarlyStopping(monitor='accuracy', patience=2) 

In [61]:
history = model.fit(X, Y, batch_size=32, epochs=50, callbacks=estop)
if estop.stopped_epoch >= 1:
    print(f'Обучение остановлено на {estop.stopped_epoch} эпохе')

Epoch 1/50
1/1 [==============================] - 0s 70ms/step - loss: 0.3155 - accuracy: 1.0000
Epoch 2/50
1/1 [==============================] - 0s 11ms/step - loss: 0.3025 - accuracy: 1.0000
Epoch 3/50
1/1 [==============================] - 0s 29ms/step - loss: 0.2904 - accuracy: 1.0000
Обучение остановлено на 2 эпохе


In [63]:
def buildPhrase(texts, str_len = 20):
  res = texts
  data = tokenizer.texts_to_sequences([texts])[0]

  for i in range(str_len):
    x = keras.utils.to_categorical(data[i: i+inp_words], num_classes=maxWordsCount) # преобразуем в One-Hot-encoding
    inp = x.reshape(1, inp_words, maxWordsCount)
    pred = model.predict( inp ) # предсказываем OHE четвертого символа
    indx = pred.argmax(axis=1)[0]
    data.append(indx)
 
    res += " " + tokenizer.index_word[indx] # дописываем строку
 
  return res

In [64]:
res = buildPhrase("позитив добавляет годы")
print(res)

ValueError: cannot reshape array of size 0 into shape (1,3,1000)

Ошибка на семинаре была в том, что мы использовать аргументом для функции строчку, которой не было файле train_data.txt

Поменяв текст на "Думайте позитивно и" и мы получим вполне ожидаемый результат

In [68]:
res = buildPhrase("Думайте позитивно и")
print(res)

1/1 [==============================] - 0s 27ms/step
Думайте позитивно и верьте в свою способность достигать отличных результатов если вы смогли в понедельник подняться с постели значит вы супер герой понедельник
